# Pruning Example
This notebook demonstrates how to use the pruning methods from this tool to compress a model. 

The example uses the MNIST dataset and a simple CNN model. The model is trained and then pruned using the methods in this tool. The pruned model is then evaluated on the test set to see how well it performs.

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

sys.path.append('../')
sys.path.append('../src/')

import src.general as general
import src.compression.pruning as prune
import src.metrics as metrics
import src.plot as plot
import src.evaluation as eval
import src.dataset_models as data
import mnist


/home/abel/anaconda3/envs/pytorch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Get device
device = general.get_device()

# Load the dataset
dataset = data.supported_datasets["MNIST"]

Using cuda: False


/home/abel/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


#### Load an existing model

In [16]:
model_state = "../models/mnist.pt"
model_class = "models.mnist"


# Load the model
model = torch.load(model_state, map_location=torch.device(device))

#### Or create a new model

In [5]:
model = mnist.MnistModel().to(device)

In [ ]:
epochs = 3

for epoch in range(1, epochs + 1):
    general.train(model, device, dataset)
    general.test(model, device, dataset)

In [ ]:
save_model = True
save_model_path = "../models/mnist.pt"

if save_model:
    torch.save(model, save_model_path)

## Pruning
Pruning is a process of reducing the size of a machine learning model by removing unimportant weights and neurons. Pruning can be used to reduce the number of parameters in a model, thereby reducing the memory footprint and the computational complexity of the model. 

Pruning is typically done in two ways: structured pruning, which involves selectively removing a larger part of the network such as a layer or a channel, and unstructured pruning, which involves removing individual weights or neurons.

### Unstructured Pruning
Here we prune individual weights or neurons.

In [17]:
# Evaluate model performance before pruning
original_results = eval.get_results(model, dataset)
plot.print_results(**original_results)

Using cuda: False


Test: 100%|██████████| 157/157 [00:01<00:00, 110.45it/s]

================================ TEST PERFORMANCE ==============================
Average loss = 0.0787
Accuracy = 0.9768
Elapsed time = 1423.28 milliseconds (9.07 per batch, 0.57 per data point)
Could not calculate FLOPS
==================================== RESULTS ===================================
Loss: 0.078731
Score: 0.976811
Time per data point: 0.5666 ms
Model Size: 1.65 MB
Number of parameters: 431080
Number of FLOPs: -1
Number of MACs: 2307728


In [18]:
# Define the parameters to prune
parameters_to_prune = (
    (model.conv1, 'weight'),
    (model.conv2, 'weight'),
    (model.fc1, 'weight'),
    (model.fc2, 'weight'),
)

prune.pruning_global(model, 0.9)

# Test model performance after pruning
pruned_results = eval.get_results(model, dataset)

plot.print_before_after_results(original_results, pruned_results)



Using cuda: False


Test: 100%|██████████| 157/157 [00:01<00:00, 114.89it/s]


================================ TEST PERFORMANCE ==============================
Average loss = 1.1647
Accuracy = 0.8695
Elapsed time = 1368.13 milliseconds (8.71 per batch, 0.54 per data point)
Could not calculate FLOPS
============================= RESULTS BEFORE & AFTER ===========================
Loss: 0.078731 -> 1.164690 (1379.34%)
Score: 0.976811 -> 0.869526 (-10.98%)
Time per data point: 0.5666 ms -> 0.5446 ms (-3.87%)
Model Size: 1.65 MB -> 1.65 MB (-0.00%)
Number of parameters: 431080 -> 431080 (-0.00%)
Number of FLOPs: -1 -> -1 (-0.00%)
Number of MACs: 2307728 -> 2307728 (-0.00%)


In [ ]:
# Print number of parameters in model
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')


# Print number of parameters pruned
total_pruned_params = sum(p.numel() for p in model.parameters() if hasattr(p, 'mask'))
print(f'{total_pruned_params:,} parameters pruned ({100 * total_pruned_params / total_params:.2f}% pruned)')

# Print number of parameters remaining
total_unpruned_params = sum(p.numel() for p in model.parameters() if not hasattr(p, 'mask'))
print(f'{total_unpruned_params:,} parameters unpruned ({100 * total_unpruned_params / total_params:.2f}% unpruned)')


431,080 total parameters.
0 parameters pruned (0.00% pruned)
431,080 parameters unpruned (100.00% unpruned)


## Structured Pruning

In [ ]:
import torch_pruning as tp
from torchvision.models import resnet18
from torchvision.models.resnet import ResNet18_Weights

# 0. prepare your model and example inputs
model = resnet18(weights=ResNet18_Weights.DEFAULT).eval()
example_inputs = torch.randn(1,3,224,224)
# input_batch = next(iter(train_loader))
# example_input = input_batch[0][0]


# 1. build dependency graph for resnet18
DG = tp.DependencyGraph().build_dependency(model, example_inputs=example_inputs)

# 2. Select some channels to prune. Here we prune the channels indexed by [2, 6, 9].
pruning_idxs = pruning_idxs=[2, 6, 9]
pruning_group = DG.get_pruning_group( model.conv1, tp.prune_conv_out_channels, idxs=pruning_idxs )

# 3. prune all grouped layer that is coupled with model.conv1
if DG.check_pruning_group(pruning_group):
    pruning_group.exec()

In [ ]:
print("After pruning:")
metrics = {
    "model": model,
    "batch_size": batch_size,
    "example_input": example_input,
}
plot.print_metrics(**metrics)

After pruning:
==================================== METRICS ===================================
Model Size: 44.62 MB
Number of parameters: 11678301


KeyError: "attribute 'flops' already exists"

In [ ]:
print(pruning_group)



--------------------------------
          Pruning Group
--------------------------------
[0] [DEP] prune_out_channels on conv1 (Conv2d(3, 61, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)) => prune_out_channels on conv1 (Conv2d(3, 61, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)), #Pruned=3
[1] [DEP] prune_out_channels on conv1 (Conv2d(3, 61, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)) => prune_out_channels on bn1 (BatchNorm2d(61, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)), #Pruned=3
[2] [DEP] prune_out_channels on bn1 (BatchNorm2d(61, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)) => prune_out_channels on _ElementWiseOp(ReluBackward0), #Pruned=3
[3] [DEP] prune_out_channels on _ElementWiseOp(ReluBackward0) => prune_out_channels on _ElementWiseOp(MaxPool2DWithIndicesBackward0), #Pruned=3
[4] [DEP] prune_out_channels on _ElementWiseOp(MaxPool2DWithIndicesBackward0) => prune_out_channels on _

In [ ]:
model = resnet18(pretrained=True)
example_inputs = torch.randn(1, 3, 224, 224)

# 0. importance criterion for parameter selections
imp = tp.importance.MagnitudeImportance(p=2, group_reduction='mean')

# 1. ignore some layers that should not be pruned, e.g., the final classifier layer.
ignored_layers = []
for m in model.modules():
    if isinstance(m, torch.nn.Linear) and m.out_features == 1000:
        ignored_layers.append(m) # DO NOT prune the final classifier!

        
# 2. Pruner initialization
iterative_steps = 5 # You can prune your model to the target sparsity iteratively.
pruner = tp.pruner.MagnitudePruner(
    model, 
    example_inputs, 
    global_pruning=False, # If False, a uniform sparsity will be assigned to different layers.
    importance=imp, # importance criterion for parameter selection
    iterative_steps=iterative_steps, # the number of iterations to achieve target sparsity
    ch_sparsity=0.5, # remove 50% channels, ResNet18 = {64, 128, 256, 512} => ResNet18_Half = {32, 64, 128, 256}
    ignored_layers=ignored_layers,
)
# print(model)
base_macs, base_nparams = tp.utils.count_ops_and_params(model, example_inputs)
for i in range(iterative_steps):
    # 3. the pruner.step will remove some channels from the model with least importance
    pruner.step()
    
    # 4. Do whatever you like here, such as fintuning
    macs, nparams = tp.utils.count_ops_and_params(model, example_inputs)
    # print(model)
    print(model(example_inputs).shape)
    print(
        "  Iter %d/%d, Params: %.2f M => %.2f M"
        % (i+1, iterative_steps, base_nparams / 1e6, nparams / 1e6)
    )
    print(
        "  Iter %d/%d, MACs: %.2f G => %.2f G"
        % (i+1, iterative_steps, base_macs / 1e9, macs / 1e9)
    )
    metrics = {
        "model": model,
        "example_input": example_input,
    }
    print(eval.get_model_size(model), "MB")
    print(len(list(model.children())), "children")
    # finetune your model here
    # finetune(model)
    # ...
print(list(model.modules()))

torch.Size([1, 1000])
  Iter 1/5, Params: 11.69 M => 9.48 M
  Iter 1/5, MACs: 1.82 G => 1.47 G
36.24 MB
10 children
torch.Size([1, 1000])
  Iter 2/5, Params: 11.69 M => 7.53 M
  Iter 2/5, MACs: 1.82 G => 1.18 G
28.8 MB
10 children
torch.Size([1, 1000])
  Iter 3/5, Params: 11.69 M => 5.82 M
  Iter 3/5, MACs: 1.82 G => 0.91 G
22.26 MB
10 children
torch.Size([1, 1000])
  Iter 4/5, Params: 11.69 M => 4.32 M
  Iter 4/5, MACs: 1.82 G => 0.68 G
16.53 MB
10 children
torch.Size([1, 1000])
  Iter 5/5, Params: 11.69 M => 3.06 M
  Iter 5/5, MACs: 1.82 G => 0.49 G
11.71 MB
10 children
[ResNet(
  (conv1): Conv2d(3, 32, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=

### Attribution Scores
Calculating the importance of different modules

In [ ]:
from torchpruner.attributions import RandomAttributionMetric  # or any of the methods above

attr = RandomAttributionMetric(model, test_loader, criterion, device)
for module in model.children():
    if len(list(module.children())) == 0:  # leaf module
        scores = attr.run(module)
        print(module)
        print (len(scores))      

Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
20
Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
50
Linear(in_features=800, out_features=500, bias=True)
500
Linear(in_features=500, out_features=10, bias=True)
10


### Any Pruning

In [ ]:
import torch
from torchvision.models import resnet18
import torch_pruning as tp

from torchvision.models.resnet import ResNet18_Weights

# 0. prepare your model and example inputs
model = resnet18(weights=ResNet18_Weights.DEFAULT).eval()

example_input = torch.randn(1,3,224,224)

# eval.get_model_flops(model, example_input)
print(eval.get_model_size(model), "MB")

# 1. build dependency graph for resnet18
DG = tp.DependencyGraph()
DG.build_dependency(model, example_inputs=example_input)

# 2. Specify the to-be-pruned channels. Here we prune those channels indexed by [2, 6, 9].
pruning_idxs = [2, 6, 9]
pruning_group = DG.get_pruning_group( model.conv1, tp.prune_conv_out_channels, idxs=pruning_idxs )

# 3. prune all grouped layer that is coupled with model.conv1 (included).
if DG.check_pruning_group(pruning_group): # avoid full pruning, i.e., channels=0.
    pruning_group.exec()


print(eval.get_model_size(model), "MB")


# 4. save & load the pruned model 
torch.save(model, 'model.pth') # save the model object
model_loaded = torch.load('model.pth') # no load_state_dict

44.66 MB
44.62 MB


In [ ]:
for x in pruning_group:
    print(x)

([DEP] prune_out_channels on conv1 (Conv2d(3, 61, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)) => prune_out_channels on conv1 (Conv2d(3, 61, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)), [2, 6, 9])
([DEP] prune_out_channels on conv1 (Conv2d(3, 61, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)) => prune_out_channels on bn1 (BatchNorm2d(61, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)), [2, 6, 9])
([DEP] prune_out_channels on bn1 (BatchNorm2d(61, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)) => prune_out_channels on _ElementWiseOp(ReluBackward0), [2, 6, 9])
([DEP] prune_out_channels on _ElementWiseOp(ReluBackward0) => prune_out_channels on _ElementWiseOp(MaxPool2DWithIndicesBackward0), [2, 6, 9])
([DEP] prune_out_channels on _ElementWiseOp(MaxPool2DWithIndicesBackward0) => prune_out_channels on _ElementWiseOp(AddBackward0), [2, 6, 9])
([DEP] prune_out_channels on _ElementWiseOp(MaxPool2DWithIndic